In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt

In [ ]:
# ----------------- Data Loading & Preprocessing -----------------
train_data = pd.read_csv("db_sc1_WiFi.csv")
test_data = pd.read_csv("Tests_Scenario1_WiFi.csv")

X_train = train_data[["RSSI A", "RSSI B", "RSSI C"]].values
y_train = train_data[["x", "y"]].values
X_test = test_data[["RSSI A", "RSSI B", "RSSI C"]].values
y_test = test_data[["x", "y"]].values

def normalize_to_0_2pi(X):
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    return 2 * np.pi * (X - X_min) / (X_max - X_min + 1e-8)

In [ ]:
X_train_norm = normalize_to_0_2pi(X_train)
X_test_norm = normalize_to_0_2pi(X_test)

X_train_t = torch.tensor(X_train_norm, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test_norm, dtype=torch.float32)

train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [ ]:
# ----------------- Quantum Circuit Setup -----------------
feature_map = ZZFeatureMap(3)
ansatz = RealAmplitudes(3, reps=3)

qc = QuantumCircuit(3)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    input_gradients=True
)

In [ ]:
# ----------------- Classical Neural Network -----------------
class ClassicalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# ----------------- Hybrid Model -----------------
class HybridModel(nn.Module):
    def __init__(self, qnn, classical_nn):
        super().__init__()
        self.qnn = TorchConnector(qnn)
        self.classical_nn = classical_nn
    
    def forward(self, x):
        q_out = self.qnn(x)
        c_out = self.classical_nn(x)
        return q_out + c_out

classical_nn = ClassicalNN()
hybrid_model = HybridModel(qnn, classical_nn)
optimizer = optim.Adam(hybrid_model.parameters(), lr=0.001)
loss_func = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)


In [ ]:
# ----------------- Training Loop -----------------
epochs = 80
training_losses = []

for epoch in range(epochs):
    epoch_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = hybrid_model(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    average_epoch_loss = epoch_loss / len(train_loader)
    training_losses.append(average_epoch_loss)
    scheduler.step(average_epoch_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {average_epoch_loss:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs + 1), training_losses, marker='o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Epochs vs Training Loss')
plt.legend()
plt.grid(True)
plt.show()
plt.savefig("wifi1_loss.jpg", dpi=300, bbox_inches='tight')

torch.save(hybrid_model.state_dict(), "hybrid_model_with_scheduler.pt")

In [ ]:
# ----------------- Fourier Surrogate Implementation -----------------
# Create grid for Fourier approximation
F_i = [3, 3, 3]
grid_points = [np.linspace(0, 2*np.pi, fi) for fi in F_i]
grid = np.meshgrid(*grid_points, indexing='ij')
grid_inputs = np.vstack([g.ravel() for g in grid]).T

# Sample quantum model outputs on the grid
grid_tensor = torch.tensor(grid_inputs, dtype=torch.float32)
quantum_outputs = []
for x in grid_tensor:
    output = hybrid_model.qnn(x.unsqueeze(0)).detach().numpy()
    quantum_outputs.append(output[0])
quantum_outputs = np.array(quantum_outputs)

if quantum_outputs.ndim == 1:
    quantum_outputs = quantum_outputs.reshape(-1, 1)
if quantum_outputs.shape[1] == 1:
    quantum_outputs = np.concatenate([quantum_outputs, quantum_outputs], axis=1)

# Build Fourier basis matrix
omega = [-1, 0, 1]
omega_combinations = np.array(np.meshgrid(omega, omega, omega)).T.reshape(-1, 3)

A = np.zeros((len(grid_inputs), len(omega_combinations)), dtype=np.complex128)
for i, x in enumerate(grid_inputs):
    for j, w in enumerate(omega_combinations):
        A[i, j] = np.exp(-1j * np.dot(w, x))

# Solve for Fourier coefficients
coefficients_x = np.linalg.lstsq(A, quantum_outputs[:, 0], rcond=None)[0]
coefficients_y = np.linalg.lstsq(A, quantum_outputs[:, 1], rcond=None)[0]

In [ ]:
# ----------------- Fourier Surrogate Model -----------------
class FourierSurrogate(nn.Module):
    def __init__(self, coefficients_x, coefficients_y, omega_combinations):
        super().__init__()
        self.coefficients_x = nn.Parameter(torch.tensor(coefficients_x, dtype=torch.complex64))
        self.coefficients_y = nn.Parameter(torch.tensor(coefficients_y, dtype=torch.complex64))
        self.omega_combinations = torch.tensor(omega_combinations, dtype=torch.float32)
    
    def forward(self, x):
        exponents = -1j * torch.einsum('bi,ji->bj', x, self.omega_combinations)
        basis = torch.exp(exponents)
        out_x = torch.matmul(basis, self.coefficients_x).real
        out_y = torch.matmul(basis, self.coefficients_y).real
        return torch.stack([out_x, out_y], dim=1)

surrogate = FourierSurrogate(coefficients_x, coefficients_y, omega_combinations)


In [ ]:
# ----------------- Surrogate Hybrid Model -----------------
class SurrogateHybridModel(nn.Module):
    def __init__(self, surrogate, classical_nn):
        super().__init__()
        self.surrogate = surrogate
        self.classical_nn = classical_nn
    
    def forward(self, x):
        x_surrogate = self.surrogate(x)
        x_classical = self.classical_nn(x)
        return x_surrogate + x_classical

surrogate_hybrid_model = SurrogateHybridModel(surrogate, hybrid_model.classical_nn)


In [ ]:
# ----------------- Evaluation -----------------
with torch.no_grad():
    surrogate_outputs = surrogate(grid_tensor).numpy()
    diff_x = np.max(np.abs(quantum_outputs[:, 0] - surrogate_outputs[:, 0]))
    diff_y = np.max(np.abs(quantum_outputs[:, 1] - surrogate_outputs[:, 1]))
    print(f"Max difference between quantum and surrogate outputs (x): {diff_x}")
    print(f"Max difference between quantum and surrogate outputs (y): {diff_y}")

Y_pred_surrogate = surrogate_hybrid_model(X_test_t).detach().numpy()
rmse_surrogate = np.sqrt(np.mean((Y_pred_surrogate - y_test)**2))
print(f"Surrogate Hybrid Model RMSE: {rmse_surrogate}")

In [ ]:
# ----------------- Evaluation -----------------
# Calculate RMSE for both models
with torch.no_grad():
    # Get predictions from both models
    Y_pred_quantum = hybrid_model(X_test_t).detach().numpy()
    Y_pred_surrogate = surrogate_hybrid_model(X_test_t).detach().numpy()

    # Calculate RMSEs
    rmse_quantum = np.sqrt(np.mean((Y_pred_quantum - y_test)**2))
    rmse_surrogate = np.sqrt(np.mean((Y_pred_surrogate - y_test)**2))

print(f"\nPerformance Comparison:")
print(f"Quantum Hybrid Model RMSE: {rmse_quantum:.4f}")
print(f"Surrogate Hybrid Model RMSE: {rmse_surrogate:.4f}")

In [ ]:
# ----------------- Plotting Section -----------------
# 1. True vs Predicted Coordinates Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_test[:, 0], Y_pred_quantum[:, 0], alpha=0.5, label='Quantum Predictions')
plt.scatter(y_test[:, 0], Y_pred_surrogate[:, 0], alpha=0.5, label='Surrogate Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('True X Position')
plt.ylabel('Predicted X Position')
plt.title('X Coordinate Predictions')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_test[:, 1], Y_pred_quantum[:, 1], alpha=0.5, label='Quantum Predictions')
plt.scatter(y_test[:, 1], Y_pred_surrogate[:, 1], alpha=0.5, label='Surrogate Predictions')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('True Y Position')
plt.ylabel('Predicted Y Position')
plt.title('Y Coordinate Predictions')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2. Error Distribution Plot
quantum_errors = np.sqrt((Y_pred_quantum[:, 0] - y_test[:, 0])**2 + 
                 (Y_pred_quantum[:, 1] - y_test[:, 1])**2)
surrogate_errors = np.sqrt((Y_pred_surrogate[:, 0] - y_test[:, 0])**2 + 
                  (Y_pred_surrogate[:, 1] - y_test[:, 1])**2)

plt.figure(figsize=(10, 6))
plt.hist(quantum_errors, bins=30, alpha=0.6, label='Quantum Model')
plt.hist(surrogate_errors, bins=30, alpha=0.6, label='Surrogate Model')
plt.xlabel('Localization Error (m)')
plt.ylabel('Frequency')
plt.title('Error Distribution Comparison')
plt.legend()
plt.grid(True)
plt.show()

# 3. Model Output Comparison Plot
plt.figure(figsize=(10, 6))
plt.scatter(Y_pred_quantum[:, 0], Y_pred_surrogate[:, 0], alpha=0.5, label='X Coordinate')
plt.scatter(Y_pred_quantum[:, 1], Y_pred_surrogate[:, 1], alpha=0.5, label='Y Coordinate')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Quantum Model Predictions')
plt.ylabel('Surrogate Model Predictions')
plt.title('Model Predictions Correlation')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# ----------------- Create Cross-Validation Set -----------------

# Combine train and test sets
X_all = np.vstack((X_train_norm, X_test_norm))
y_all = np.vstack((y_train, y_test))

# Get sample sizes
n_train_cv = int(0.6 * len(X_train_norm))
n_test_cv = int(0.4 * len(X_test_norm))

# Split 60% from training and 40% from test sets
X_train_cv, _, y_train_cv, _ = train_test_split(X_train_norm, y_train, train_size=n_train_cv, random_state=42)
X_test_cv, _, y_test_cv, _ = train_test_split(X_test_norm, y_test, train_size=n_test_cv, random_state=42)

# Combine for cross-validation
X_cv = np.vstack((X_train_cv, X_test_cv))
y_cv = np.vstack((y_train_cv, y_test_cv))

# Convert to torch tensors
X_cv_t = torch.tensor(X_cv, dtype=torch.float32)
y_cv_t = torch.tensor(y_cv, dtype=torch.float32)

# ----------------- Evaluation on Cross-Validation Data -----------------

# HQNN Evaluation
with torch.no_grad():
    y_cv_pred_qnn = hybrid_model(X_cv_t).numpy()
    rmse_cv_qnn = np.sqrt(np.mean((y_cv_pred_qnn - y_cv)**2))
    print(f"\nHQNN Hybrid Model RMSE on CV Data: {rmse_cv_qnn:.4f}")

# Surrogate Evaluation
with torch.no_grad():
    y_cv_pred_surrogate = surrogate_hybrid_model(X_cv_t).numpy()
    rmse_cv_surrogate = np.sqrt(np.mean((y_cv_pred_surrogate - y_cv)**2))
    print(f"Surrogate Hybrid Model RMSE on CV Data: {rmse_cv_surrogate:.4f}")


In [ ]:
# ----------------- Error Bar Analysis with Complete Test Set -----------------
# First, ensure we have predictions for the full test set
with torch.no_grad():
    Y_pred_quantum = hybrid_model(X_test_t).detach().numpy()
    Y_pred_surrogate = surrogate_hybrid_model(X_test_t).detach().numpy()

# Calculate errors for the complete test dataset
quantum_x_errors = np.abs(Y_pred_quantum[:, 0] - y_test[:, 0])
quantum_y_errors = np.abs(Y_pred_quantum[:, 1] - y_test[:, 1])
surrogate_x_errors = np.abs(Y_pred_surrogate[:, 0] - y_test[:, 0])
surrogate_y_errors = np.abs(Y_pred_surrogate[:, 1] - y_test[:, 1])

# Create error bar plot for the complete test dataset
plt.figure(figsize=(14, 8))

# X coordinate error bars
plt.subplot(1, 2, 1)
plt.errorbar(np.arange(len(y_test)), Y_pred_quantum[:, 0], 
             yerr=quantum_x_errors, fmt='o', label='HQNN Model', 
             capsize=3, elinewidth=0.5, alpha=0.5)
plt.errorbar(np.arange(len(y_test)) + 0.3, Y_pred_surrogate[:, 0], 
             yerr=surrogate_x_errors, fmt='s', label='Surrogate Model', 
             capsize=3, elinewidth=0.5, alpha=0.5)
plt.scatter(np.arange(len(y_test)) + 0.15, y_test[:, 0], 
            color='red', marker='*', s=20, label='Ground Truth')
plt.xlabel('Sample Index')
plt.ylabel('X Coordinate')
plt.title('X Coordinate Predictions with Error Bars (Full Test Set)')
plt.legend()
plt.grid(True)

# Y coordinate error bars
plt.subplot(1, 2, 2)
plt.errorbar(np.arange(len(y_test)), Y_pred_quantum[:, 1], 
             yerr=quantum_y_errors, fmt='o', label='HQNN Model', 
             capsize=3, elinewidth=0.5, alpha=0.5)
plt.errorbar(np.arange(len(y_test)) + 0.3, Y_pred_surrogate[:, 1], 
             yerr=surrogate_y_errors, fmt='s', label='Surrogate Model', 
             capsize=3, elinewidth=0.5, alpha=0.5)
plt.scatter(np.arange(len(y_test)) + 0.15, y_test[:, 1], 
            color='red', marker='*', s=20, label='Ground Truth')
plt.xlabel('Sample Index')
plt.ylabel('Y Coordinate')
plt.title('Y Coordinate Predictions with Error Bars (Full Test Set)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# ----------------- Frequency Pruning Analysis -----------------
# Get the magnitudes of coefficients
coef_x_magnitude = np.abs(surrogate.coefficients_x.detach().numpy())
coef_y_magnitude = np.abs(surrogate.coefficients_y.detach().numpy())

# Combined magnitude for sorting
combined_magnitude = coef_x_magnitude + coef_y_magnitude

# Sort indices by magnitude (from lowest to highest)
sorted_indices = np.argsort(combined_magnitude)
num_coeffs = len(sorted_indices)

# Prepare for tracking results
pruning_percentages = []
surrogate_rmse_values = []
num_iterations = 9  # Remove 10% each time for 9 iterations (up to 90%)

# Store HQNN baseline RMSE
hqnn_rmse = rmse_quantum

# Plot the original frequency distribution
plt.figure(figsize=(10, 6))
plt.bar(np.arange(num_coeffs), combined_magnitude[sorted_indices], alpha=0.7)
plt.xlabel('Sorted Frequency Index')
plt.ylabel('Coefficient Magnitude')
plt.title('Frequency Distribution of Surrogate Model (Sorted by Magnitude)')
plt.grid(True)
plt.savefig("wifi1_freq_dist.jpg", dpi=300, bbox_inches='tight')
plt.show()


# Iteratively prune and evaluate
for i in range(num_iterations):
    # Calculate how many coefficients to keep (as percentage)
    keep_percentage = 100 - 10 * (i + 1)
    pruning_percentages.append(keep_percentage)
    
    # Calculate how many coefficients to keep
    keep_count = int(keep_percentage/100 * num_coeffs)
    
    # Determine how many to remove from each side
    remove_per_side = (num_coeffs - keep_count) // 2
    
    # Select the middle frequencies (remove equal amounts from both ends)
    kept_indices = sorted_indices[remove_per_side:num_coeffs-remove_per_side]
    
    # Create a pruned surrogate model
    pruned_coeffs_x = np.zeros_like(coef_x_magnitude, dtype=np.complex128)
    pruned_coeffs_y = np.zeros_like(coef_y_magnitude, dtype=np.complex128)
    
    # Only keep selected frequencies
    for idx in kept_indices:
        pruned_coeffs_x[idx] = surrogate.coefficients_x.detach().numpy()[idx]
        pruned_coeffs_y[idx] = surrogate.coefficients_y.detach().numpy()[idx]
    
    # Create pruned surrogate model
    pruned_surrogate = FourierSurrogate(pruned_coeffs_x, pruned_coeffs_y, omega_combinations)
    pruned_surrogate_hybrid = SurrogateHybridModel(pruned_surrogate, hybrid_model.classical_nn)
    
    # Evaluate
    with torch.no_grad():
        Y_pred_pruned = pruned_surrogate_hybrid(X_test_t).detach().numpy()
        rmse_pruned = np.sqrt(np.mean((Y_pred_pruned - y_test)**2))
        surrogate_rmse_values.append(rmse_pruned)
    
    # Print progress
    print(f"Kept {keep_percentage}% of frequencies ({len(kept_indices)}/{num_coeffs}), RMSE: {rmse_pruned:.4f}")

# Plot the RMSE vs pruning percentage
plt.figure(figsize=(10, 6))
plt.plot(pruning_percentages, surrogate_rmse_values, 'o-', label='Pruned Surrogate Model')
plt.axhline(y=hqnn_rmse, color='r', linestyle='--', label='HQNN Model')
plt.xlabel('Percentage of Frequencies Kept')
plt.ylabel('RMSE')
plt.title('Effect of Frequency Pruning on Model Accuracy')
plt.legend()
plt.grid(True)
plt.xticks(pruning_percentages)
plt.gca().invert_xaxis()  # Show decreasing percentages from left to right
plt.show()

